# Estimación de dirección de arribo de fuentes sonoras 

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

### Nombre de la simulación

In [2]:
#Nombre de la simulación
simulation_name = "room_big_anec_corto"

### Configuración de Gspreadsheet

In [3]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.worksheet(simulation_name)

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

WorksheetNotFound: room_big_anec_corto

## Bloque de código para ejecutar todo directamente

Permite cargar los datos del Excel y procesa todo en tiempo real. Acá se puede elegir si guardar o no los audios con **save_audio = True**, en caso de ser falso se trabajará todo con numpy array con los datos de las señales procesadas.

In [ ]:
# Iterar sobre el DataFrame 
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{simulation_name}/{sim["sim_name"]}',
        sim_name=sim["sim_name"],
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=True
    )
    
    avg_angle, avg_tdoa, angles, tdoas = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en cada fila individual

    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]
    df_simulations.at[idx, 't_12'] = tdoas[0]
    df_simulations.at[idx, 't_13'] = tdoas[1]
    df_simulations.at[idx, 't_14'] = tdoas[2]

# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations.head()



DataFrame guardado correctamente en Google Sheets.


,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,avg_angle_deg,theta_1,theta_2,theta_3,t_12,t_13,t_14,avg_tdoa,room_dim,error
0,room_big_anec_corto_0,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,4.024728e-07,0.000000,0.000000,0.000001,0.000292,0.000583,0.000875,0.000583,"100,100,100",
1,room_big_anec_corto_5,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,4.024728e-07,0.000000,0.000000,0.000001,0.000292,0.000583,0.000875,0.000583,"100,100,100",
2,room_big_anec_corto_10,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,4.175968e+00,0.000000,0.000000,12.527905,0.000292,0.000583,0.000854,0.000576,"100,100,100",
3,room_big_anec_corto_15,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,1.647291e+01,21.726984,15.271729,12.420011,0.000271,0.000562,0.000854,0.000562,"100,100,100",
4,room_big_anec_corto_20,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,2.172698e+01,21.726984,21.726984,21.726984,0.000271,0.000542,0.000812,0.000542,"100,100,100",
5,room_big_anec_corto_25,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,2.453437e+01,21.726984,26.718130,25.158004,0.000271,0.000521,0.000792,0.000528,"100,100,100",
6,room_big_anec_corto_30,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,3.096297e+01,30.962969,30.962969,30.962969,0.000250,0.000500,0.000750,0.000500,"100,100,100",
7,room_big_anec_corto_35,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,3.627935e+01,38.182877,34.737544,35.917636,0.000229,0.000479,0.000708,0.000472,"100,100,100",
8,room_big_anec_corto_40,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,3.996831e+01,38.182877,41.382545,40.339502,0.000229,0.000438,0.000667,0.000444,"100,100,100",
9,room_big_anec_corto_45,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,4.439094e+01,44.390938,44.390938,44.390938,0.000208,0.000417,0.000625,0.000417,"100,100,100",


# Bloque de código para trabajar con los audios ya creados

In [ ]:
# Nombre de la simulación
folder_wav = "room_big_sweep"
simulation_name = "room_big_sweep_roth"

# Configurar credenciales para acceder a Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.worksheet(simulation_name)

# Leer datos de la hoja de cálculo
df_simulations = pd.DataFrame(sheet.get_all_records())
df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego

# directorio base donde se guardan las simulaciones
base_audio_dir = f"audios/output/{folder_wav}"

# Iterar sobre el DataFrame y procesar cada simulación
for idx, sim in df_simulations.iterrows():
    sim_path = os.path.join(base_audio_dir, sim["sim_name"])  # Directorio de la simulación

    # Obtener todos los archivos de micrófonos en la simulación
    mic_files = sorted(glob.glob(os.path.join(sim_path, "*.wav")))
    print(mic_files)
    # Opción 1: Pasar rutas de archivos directamente
    avg_angle, avg_tdoa, angles, tdoas = estimation.estimate_doa(
        signals=mic_files,  # Pasamos las rutas de archivos directamente
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en el DataFrame
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]
    df_simulations.at[idx, 't_12'] = tdoas[0]
    df_simulations.at[idx, 't_13'] = tdoas[1]
    df_simulations.at[idx, 't_14'] = tdoas[2]

# Convertir el DataFrame a formato lista para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir los datos actualizados en Google Sheets
sheet.update(data)

print("Resultados de DOA guardados correctamente en Google Sheets.")
df_simulations.head()

['audios/output/room_big_sweep\\room_big_sine_sweep_0\\mic_1_room_big_sine_sweep_0.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_0\\mic_2_room_big_sine_sweep_0.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_0\\mic_3_room_big_sine_sweep_0.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_0\\mic_4_room_big_sine_sweep_0.wav']
['audios/output/room_big_sweep\\room_big_sine_sweep_5\\mic_1_room_big_sine_sweep_5.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_5\\mic_2_room_big_sine_sweep_5.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_5\\mic_3_room_big_sine_sweep_5.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_5\\mic_4_room_big_sine_sweep_5.wav']
['audios/output/room_big_sweep\\room_big_sine_sweep_10\\mic_1_room_big_sine_sweep_10.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_10\\mic_2_room_big_sine_sweep_10.wav', 'audios/output/room_big_sweep\\room_big_sine_sweep_10\\mic_3_room_big_sine_sweep_10.wav', 'audios/output/room_big

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,avg_angle_deg,theta_1,theta_2,theta_3,t_12,t_13,t_14,avg_tdoa,room_dim,error
0,room_big_sine_sweep_0,roth,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,16.472908,21.726984,15.271729,12.420011,0.000271,0.000562,0.000854,0.000562,"100,100,100",
1,room_big_sine_sweep_5,roth,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,16.472908,21.726984,15.271729,12.420011,0.000271,0.000562,0.000854,0.000562,"100,100,100",
2,room_big_sine_sweep_10,roth,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,18.225595,21.726984,15.271729,17.67807,0.000271,0.000562,0.000833,0.000556,"100,100,100",
3,room_big_sine_sweep_15,roth,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,23.456008,30.962969,21.726984,17.67807,0.00025,0.000542,0.000833,0.000542,"100,100,100",
4,room_big_sine_sweep_20,roth,sine_sweep_24bit.wav,100,100,100,3,70,4,0.1,...,27.613034,30.962969,26.71813,25.158004,0.00025,0.000521,0.000792,0.000521,"100,100,100",


: 